# Glandu segmentācijas datu sagatavošana U-Net un FPN (segmentation_models_pytorch)

Šajā notebook'ā mēs:

- apskatīsim datu struktūru glandu segmentācijas datu kopai,
- nolasīsim anotāciju JSON datnes un pārvērtīsim tās par bināru masku,
- izveidosim `torch.utils.data.Dataset` klasi,
- vizualizēsim dažus piemērus (attēls + maskas pārklājums),
- sagatavosim visu tā, lai nākamajos piezīmjdatoros varētu:
  - trenēt U-Net un FPN modeļus,
  - salīdzināt to rezultātus.

No sākuma datu struktūra ir šāda:

``` bash
data/
  training/
    img/*.bmp
    ann/*.bmp.json
  test/
    img/*.bmp
    ann/*.bmp.json
```

In [ ]:
# Importējam nepieciešamas bibliotēkas
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import json

import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset

In [ ]:
# Saknes ceļš uz glandu datu kopu (pielāgo, ja vajag)
GLAND_ROOT = Path("data/gland_seg")

TRAIN_DIR = GLAND_ROOT / "training"
TEST_DIR = GLAND_ROOT / "test"

print("Treniņa mape:", TRAIN_DIR)
print("Testa mape:", TEST_DIR)

In [ ]:
# Atradīsim kādu vienu anotācijas datni, lai apskatītos struktūru
sample_ann_files = list((TRAIN_DIR / "ann").glob("*.bmp.json"))
len(sample_ann_files), sample_ann_files[:3]

In [ ]:
# Apskatām pirmo anotāciju (strukturāli)
sample_ann_path = sample_ann_files[0]
print("Parauga anotācijas datne:", sample_ann_path)

with open(sample_ann_path, "r", encoding="utf-8") as f:
    sample_ann = json.load(f)

sample_ann.keys(), sample_ann.get("size"), len(sample_ann.get("objects", []))

In [ ]:
# Apskatām pirmo objektu paraugā
first_obj = sample_ann["objects"][0]
first_obj

In [ ]:
# Apskatām, kādas classTitle klases parādās datu kopā (pirmajiem N failiem)
unique_classes = set()

for ann_path in sample_ann_files[:50]:  # var palielināt, ja gribas pilnīgāk
    with open(ann_path, "r", encoding="utf-8") as f:
        ann = json.load(f)
    for obj in ann.get("objects", []):
        unique_classes.add(obj.get("classTitle", "UNKNOWN"))

unique_classes

In [ ]:
def create_binary_mask_from_json(json_path: Path) -> np.ndarray:
    """
    No anotācijas JSON (Supervisely tipa poligoni) izveido bināru masku.
    Rezultāts: maska ar izmēru (H, W), kur 1 = glands, 0 = fons.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    h = data["size"]["height"]
    w = data["size"]["width"]

    mask = np.zeros((h, w), dtype=np.uint8)

    for obj in data.get("objects", []):
        # Ja gribam filtrēt pēc konkrētās klases, var te ielikt if
        # if obj["classTitle"] not in ["gland", ...]: continue

        points = obj["points"]["exterior"]  # saraksts ar [x, y] punktiem
        if len(points) < 3:
            continue  # poligons ar < 3 punktiem nav jēdzīgs

        # OpenCV fillPoly sagaida int32 masīvu formā (N,1,2)
        pts = np.array(points, dtype=np.int32)
        pts = pts.reshape((-1, 1, 2))

        cv2.fillPoly(mask, [pts], 1)  # aizpildām poligonu ar vērtību 1

    return mask

In [ ]:
# Izvēlamies vienu attēlu+anotāciju no training
img_name = sample_ann_path.stem        # "12345.bmp"
sample_img_path = TRAIN_DIR / "img" / img_name
print("Parauga attēls:", sample_img_path)

img = Image.open(sample_img_path).convert("RGB")
mask = create_binary_mask_from_json(sample_ann_path)

img_np = np.array(img)
print("Attēla forma:", img_np.shape)
print("Maskas forma:", mask.shape, "unikālās vērtības:", np.unique(mask))

In [ ]:
# Vizualizējam attēlu + masku
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.title("Oriģinālais attēls")
plt.imshow(img_np)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Maska (bināra)")
plt.imshow(mask, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Pārklājums (attēls + maska)")
overlay = img_np.copy()
# padarīsim masku sarkanu
red = overlay.copy()
red[..., 0] = 255
red[..., 1] = 0
red[..., 2] = 0

alpha = 0.5
mask_3c = np.stack([mask]*3, axis=-1).astype(bool)
overlay[mask_3c] = (alpha * red[mask_3c] + (1 - alpha) * overlay[mask_3c]).astype(np.uint8)

plt.imshow(overlay)
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
class GlandSegmentationDataset(Dataset):
    """
    PyTorch Dataset glandu segmentācijai.
    Pieņem datu struktūru:

      root/
        training/
          img/*.bmp
          ann/*.bmp.json
        test/
          img/*.bmp
          ann/*.bmp.json
    """

    def __init__(self, root_dir: Path, split: str = "training", transform: Optional = None):
        assert split in ["training", "test"], "split jābūt 'training' vai 'test'"
        self.root_dir = Path(root_dir)
        self.split = split
        self.transform = transform

        img_dir = self.root_dir / split / "img"
        ann_dir = self.root_dir / split / "ann"

        self.samples: List[Tuple[Path, Path]] = []

        for img_path in sorted(img_dir.glob("*.bmp")):
            # "12345.bmp" -> "12345.bmp.json"
            ann_path = ann_dir / f"{img_path.name}.json"
            if not ann_path.exists():
                # print(f"[BRĪDINĀJUMS] Trūkst anotācijas priekš {img_path.name}")
                continue
            self.samples.append((img_path, ann_path))

        if not self.samples:
            raise RuntimeError(f"Neatradu nevienu attēla+anotācijas pāri mapē {img_dir}")

        print(f"GlandSegmentationDataset ({split}): {len(self.samples)} paraugi.")

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int):
        img_path, ann_path = self.samples[idx]

        img = Image.open(img_path).convert("RGB")
        img_np = np.array(img, dtype=np.float32) / 255.0  # (H,W,3)

        mask_np = create_binary_mask_from_json(ann_path).astype(np.float32)  # (H,W)

        if self.transform is not None:
            transformed = self.transform(image=img_np, mask=mask_np)
            img_np = transformed["image"]
            mask_np = transformed["mask"]

        img_tensor = torch.from_numpy(img_np).permute(2, 0, 1)  # (3,H,W)
        mask_tensor = torch.from_numpy(mask_np).unsqueeze(0)    # (1,H,W)

        return img_tensor, mask_tensor, img_path.name

In [ ]:
train_ds = GlandSegmentationDataset(GLAND_ROOT, split="training", transform=None)
test_ds = GlandSegmentationDataset(GLAND_ROOT, split="test", transform=None)

len(train_ds), len(test_ds)

In [ ]:
# Paņemam dažus paraugus no treniņa kopas
n_show = 3
fig, axs = plt.subplots(n_show, 3, figsize=(12, 4 * n_show))

for i in range(n_show):
    img_t, mask_t, fname = train_ds[i]
    img_np = img_t.permute(1, 2, 0).numpy()       # (H,W,3)
    mask_np = mask_t.squeeze(0).numpy()           # (H,W)

    axs[i, 0].imshow(img_np)
    axs[i, 0].set_title(f"{fname} - attēls")
    axs[i, 0].axis("off")

    axs[i, 1].imshow(mask_np, cmap="gray")
    axs[i, 1].set_title("Maska (0/1)")
    axs[i, 1].axis("off")

    # overlay
    overlay = img_np.copy()
    red = np.zeros_like(overlay)
    red[..., 0] = 1.0
    alpha = 0.5
    mask3 = np.stack([mask_np]*3, axis=-1).astype(bool)
    overlay[mask3] = alpha * red[mask3] + (1 - alpha) * overlay[mask3]

    axs[i, 2].imshow(overlay)
    axs[i, 2].set_title("Pārklājums (attēls + maska)")
    axs[i, 2].axis("off")

plt.tight_layout()
plt.show()

## Kopsavilkums

- Datu kopa ir nolasīta no BMP attēliem un JSON anotācijām.
- No anotācijām ģenerējam **bināru masku**: `1 = gland`, `0 = fons`.
- Izveidota `GlandSegmentationDataset` klase, kas atgriež:
  - `img_tensor`: `(3, H, W)`, `float32`, normalizēts [0,1],
  - `mask_tensor`: `(1, H, W)`, `float32`, vērtības 0/1,
  - faila nosaukumu, ko var izmantot debugam.

Nākamais solis – **treniņa notebook**, kurā mēs:

- izveidosim `DataLoader` treniņam un validācijai,
- uztrenēsim `segmentation_models_pytorch.Unet` un `segmentation_models_pytorch.FPN`,
- salīdzināsim to kvalitāti (loss, IoU/Dice, vizuālie rezultāti).